In [131]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CompactSmallFiles") \
    .getOrCreate()

# Đọc toàn bộ dữ liệu đã ghi từ streaming (dưới dạng CSV)
df = spark.read.option("header", "false") \
    .csv("hdfs://localhost:9000/transactions_by_hour")

# Gộp lại thành 1 file (hoặc n file nếu dùng repartition(n))
df.coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header", "false") \
    .csv("hdfs://localhost:9000/transactions_by_hour_compacted")

spark.stop()

25/06/26 16:34:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appNasme("MergeCSV").getOrCreate()

# Đường dẫn folder chứa output CSV stream
output_path = "/path/to/stream_output/"

# --- 1. Append mode ---
# Đọc tất cả file CSV trong thư mục output, gộp lại 1 DataFrame duy nhất
df_append = spark.read.option("header", True).csv(output_path)
# Nếu muốn ghi lại thành 1 file CSV
df_append.coalesce(1).write.mode("overwrite").option("header", True).csv("/path/to/final_output/append_merged")

# --- 2. Complete mode ---
# Thường output sẽ có nhiều thư mục con batch-xxxxx
# Lấy thư mục con mới nhất (giả sử tên thư mục dạng batch-00001, batch-00002, ...)
import os
all_subdirs = [d for d in os.listdir(output_path) if os.path.isdir(os.path.join(output_path, d))]
latest_subdir = sorted(all_subdirs)[-1]  # lấy thư mục cuối cùng theo tên
latest_path = os.path.join(output_path, latest_subdir)

df_complete = spark.read.option("header", True).csv(latest_path)
df_complete.coalesce(1).write.mode("overwrite").option("header", True).csv("/path/to/final_output/complete_merged")

# --- 3. Update mode ---
# Đọc tất cả file CSV như append, nhưng có thể cần xử lý trùng bản ghi nếu có key
df_update = spark.read.option("header", True).csv(output_path)

# Giả sử bạn có cột 'id' làm key, giữ bản ghi mới nhất theo timestamp hoặc batch_id
# Ví dụ giữ record cuối cùng cho mỗi 'id' theo timestamp (nếu có)
# df_update = df_update.withColumn("timestamp", col("timestamp").cast("timestamp"))
# window_spec = Window.partitionBy("id").orderBy(col("timestamp").desc())
# df_latest = df_update.withColumn("rank", row_number().over(window_spec)).filter(col("rank") == 1).drop("rank")

# Nếu không có key, chỉ gộp hết
df_update.coalesce(1).write.mode("overwrite").option("header", True).csv("/path/to/final_output/update_merged")

spark.stop()


In [116]:
from hdfs import InsecureClient
import pandas as pd
import requests
import json
import numpy as np
import os


In [118]:
time_path = "last_push_time.txt"

# Tạo file nếu chưa có, mặc định là 1970
if not os.path.exists(time_path):
    with open(time_path, "w") as f:
        f.write("1970-01-01T00:00:00Z")

In [119]:
# Kết nối đến HDFS
client = InsecureClient('http://localhost:9870', user='panda')

# Thư mục chứa file output Spark
folder_path = '/transactions_by_hour_compacted/'


In [132]:
# Lấy danh sách file trong thư mục
files = client.list(folder_path)

# Lọc ra file .csv chứa chữ 'part'
csv_files = [f for f in files if f.endswith('.csv') and 'part' in f]

# Sắp xếp để lấy file mới nhất
csv_files.sort(reverse=True)

# Kiểm tra file
if not csv_files:
    raise Exception("Không tìm thấy file CSV phù hợp.")

# Lấy tên file đầu tiên
latest_file = csv_files[0]
print("📄 File được chọn:", latest_file)


📄 File được chọn: part-00000-f08f66de-b119-4c10-8a5f-7e6ac462f922-c000.csv


In [133]:
# Đọc file CSV từ HDFS
with client.read(folder_path + latest_file, encoding='utf-8') as reader:
    df = pd.read_csv(reader, header=0)

# Đọc mốc thời gian cuối
with open(time_path, "r") as f:
    last_push_time = pd.to_datetime(f.read().strip())

# # Chuyển cột event_time về datetime
# df["event_time"] = pd.to_datetime(df["event_time"])

# Hiển thị vài dòng dữ liệu
df.head(-5)


,User,Card,event_time,Amount_casted,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors,Is Fraud
0,0,0,2022-01-01T07:59:00.000+07:00,81.46,Chip Transaction,Shell,La Verne,NV,90001.0,7538.0,NaN,No
1,0,0,2022-01-01T10:27:00.000+07:00,76.63,Chip Transaction,Best Buy,Los Angeles,TX,90001.0,5651.0,NaN,No
2,0,0,2022-01-01T15:00:00.000+07:00,118.16,Chip Transaction,7-Eleven,Los Angeles,TX,91750.0,5651.0,Transaction Timeout,Yes
3,0,0,2022-01-01T17:26:00.000+07:00,53.81,Swipe Transaction,Shell,Pasadena,CA,90001.0,5411.0,Account Closed,Yes
4,0,0,2022-01-01T20:12:00.000+07:00,6.4,Online Transaction,Best Buy,La Verne,NV,91752.0,5651.0,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...
225,User,Card,event_time,Amount_casted,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors,Is Fraud
226,0,0,2022-01-18T03:59:00.000+07:00,102.11,Chip Transaction,Shell,La Verne,CA,91750.0,4900.0,NaN,No
227,User,Card,event_time,Amount_casted,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors,Is Fraud
228,0,0,2022-01-18T08:07:00.000+07:00,80.0,Chip Transaction,Costco,Mira Loma,CA,90001.0,5651.0,NaN,No


In [134]:
rows = []
print(last_push_time)
for _, row in df.iterrows():
    try:
        user_val = int(row["User"])
    except ValueError:
        # print(row)
        continue  # bỏ qua dòng lỗi
    # print(pd.to_datetime(row["event_time"], utc=True))
    if pd.to_datetime(row["event_time"], utc=True) > last_push_time:
        rows.append({
            "User": int(row["User"]),
            "Card": int(row["Card"]),
            "event_time": row["event_time"],
            "Amount": float(row["Amount_casted"]),
            "Use Chip": str(row["Use Chip"]),
            "Merchant Name": str(row["Merchant Name"]),
            "Merchant City": str(row["Merchant City"]),
            "Merchant State": str(row["Merchant State"]),
            "Zip": float(row["Zip"]),
            "MCC": float(row["MCC"]) ,
            "Errors": "" if pd.isna(row["Errors"]) else str(row["Errors"]),
            "Is Fraud": str(row["Is Fraud"])
        })

print(f"Tổng số dòng cần push: {len(rows)}")


2022-01-14 02:33:00+07:00
Tổng số dòng cần push: 39


In [ ]:
# Đổi link này thành link thực tế của bạn
push_url = "https://api.powerbi.com/beta/40127cd4-45f3-49a3-b05d-315a43a9f033/datasets/88dcb9eb-c4d0-4f67-8b9a-d0c4a1ae501c/rows?experience=power-bi&key=uLrnDuq85qTeWWWfTmZy5owDgmqeBI3m4Ef%2FP3AEZ9ZpHJrPx33S7AAFB5Sy3ucYxmDFr9gqS83%2F9UxACGm1Pg%3D%3D"

headers = {'Content-Type': 'application/json'}

# Đẩy dữ liệu theo batch
batch_size = 100
for i in range(0, len(rows), batch_size):
    batch = rows[i:i + batch_size]
    res = requests.post(push_url, headers=headers, data=json.dumps(batch))
    print(f"✅ Batch {i} status: {res.status_code}")

if rows:
    # Ép về datetime để gọi .isoformat()
    latest_time = max([pd.to_datetime(r["event_time"]) for r in rows])

    # Ghi vào file
    with open("last_push_time.txt", "w") as f:
        f.write(latest_time.isoformat())


In [135]:
if rows:
    # Ép về datetime để gọi .isoformat()
    latest_time = max([pd.to_datetime(r["event_time"]) for r in rows])

    # Ghi vào file
    with open("last_push_time.txt", "w") as f:
        f.write(latest_time.isoformat())